In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb

# Loading the Dataset

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/prajwalp111/data/refs/heads/main/vgsales.csv')
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


# Pre-Processing

In [3]:
df.shape


(16598, 11)

In [4]:
missing_values = df.isnull().sum()
missing_percent = (missing_values/len(df))*100
print("missing values______________ \n",missing_values)
print("missing % _______________\n",missing_percent)

missing values______________ 
 Rank              0
Name              0
Platform          0
Year            271
Genre             0
Publisher        58
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64
missing % _______________
 Rank            0.000000
Name            0.000000
Platform        0.000000
Year            1.632727
Genre           0.000000
Publisher       0.349440
NA_Sales        0.000000
EU_Sales        0.000000
JP_Sales        0.000000
Other_Sales     0.000000
Global_Sales    0.000000
dtype: float64


In [5]:
#df['Year'].fillna(df['Year'].median(),inplace=True)
df['Year'] = df['Year'].fillna(df['Year'].median())

In [6]:
df = df.dropna(subset=['Publisher'])

In [7]:
missing_values = df.isnull().sum()
missing_percent = (missing_values/len(df))*100
print("missing values \n",missing_values)
print("missing % \n",missing_percent)

missing values 
 Rank            0
Name            0
Platform        0
Year            0
Genre           0
Publisher       0
NA_Sales        0
EU_Sales        0
JP_Sales        0
Other_Sales     0
Global_Sales    0
dtype: int64
missing % 
 Rank            0.0
Name            0.0
Platform        0.0
Year            0.0
Genre           0.0
Publisher       0.0
NA_Sales        0.0
EU_Sales        0.0
JP_Sales        0.0
Other_Sales     0.0
Global_Sales    0.0
dtype: float64


# Data Visualization

In [8]:
genre_sales = df.groupby('Genre')['Global_Sales'].sum().sort_values(ascending=False)
print(genre_sales)

# df.groupby('Genre') -> grups dataset of Genre column
# ['Global_Sales'].sum() -> For each genre it calculates the total Global_sales by summing the 'Global_Sales' values.
# .sort_values(ascending=False) -> Sorts the total sales from highest to lowest (most popular genre at the top).

Genre
Action          1750.33
Sports          1329.35
Shooter         1037.32
Role-Playing     927.31
Platform         830.87
Misc             802.15
Racing           732.03
Fighting         445.70
Simulation       391.96
Puzzle           244.94
Adventure        238.77
Strategy         174.83
Name: Global_Sales, dtype: float64
